# Classifictation model 

In [2]:
%reset -f

In [3]:
# Libraries

import os
from PIL import Image
import itertools

import cv2
from skimage import io, color, filters
from skimage.feature import local_binary_pattern
import numpy as np 
import pandas as pd 
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator


import warnings
warnings.filterwarnings('ignore')

print('Modules Successfully Loaded')

c:\Users\JALAL\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,



Modules Successfully Loaded


## Dataset 

In [4]:
data_dir= "./Multi-class Weather Dataset"
paths = []
labels = []
folds = os.listdir(data_dir)
for fold in folds :
    condition_path = data_dir +"/" + fold
    all_pic = os.listdir(condition_path)
    for each_pic in all_pic:
        each_pic_path = condition_path +"/" + each_pic
        paths.append(each_pic_path)
        labels.append(fold)
        
pseries = pd.Series(paths, name='Picture Path')
lseries = pd.Series(labels, name='Label')

df = pd.concat([pseries, lseries], axis =1)
df.head()

,Picture Path,Label
0,./Multi-class Weather Dataset/Cloudy/cloudy1.jpg,Cloudy
1,./Multi-class Weather Dataset/Cloudy/cloudy10.jpg,Cloudy
2,./Multi-class Weather Dataset/Cloudy/cloudy100...,Cloudy
3,./Multi-class Weather Dataset/Cloudy/cloudy101...,Cloudy
4,./Multi-class Weather Dataset/Cloudy/cloudy102...,Cloudy


In [5]:
# Convert the labels to French
df['Label'] = df['Label'].replace({'Cloudy': 'Nuageux', 'Rain': 'Pluvieux', 'Shine': 'Ensoleillé', 'Sunrise': 'Lever du soleil'})
df

,Picture Path,Label
0,./Multi-class Weather Dataset/Cloudy/cloudy1.jpg,Nuageux
1,./Multi-class Weather Dataset/Cloudy/cloudy10.jpg,Nuageux
2,./Multi-class Weather Dataset/Cloudy/cloudy100...,Nuageux
3,./Multi-class Weather Dataset/Cloudy/cloudy101...,Nuageux
4,./Multi-class Weather Dataset/Cloudy/cloudy102...,Nuageux
...,...,...
1120,./Multi-class Weather Dataset/Sunrise/sunrise9...,Lever du soleil
1121,./Multi-class Weather Dataset/Sunrise/sunrise9...,Lever du soleil
1122,./Multi-class Weather Dataset/Sunrise/sunrise9...,Lever du soleil
1123,./Multi-class Weather Dataset/Sunrise/sunrise9...,Lever du soleil


In [6]:

# Function to get image dimensions
def get_image_dimensions(image_path):
    with Image.open(image_path) as img:
        width, height = img.size
    return width, height 

# Add columns to dataframe
df['width'], df['height'] = zip(*df['Picture Path'].map(get_image_dimensions))

df.head()

,Picture Path,Label,width,height
0,./Multi-class Weather Dataset/Cloudy/cloudy1.jpg,Nuageux,600,400
1,./Multi-class Weather Dataset/Cloudy/cloudy10.jpg,Nuageux,271,186
2,./Multi-class Weather Dataset/Cloudy/cloudy100...,Nuageux,259,194
3,./Multi-class Weather Dataset/Cloudy/cloudy101...,Nuageux,338,149
4,./Multi-class Weather Dataset/Cloudy/cloudy102...,Nuageux,800,377


# Train Test Split (80% - 20%)

In [7]:
train_set, test_set = train_test_split(df, test_size=0.2, shuffle=True, stratify=df["Label"], random_state=42)

In [8]:
train_set.head()

,Picture Path,Label,width,height
646,./Multi-class Weather Dataset/Shine/shine217.jpg,Ensoleillé,270,169
892,./Multi-class Weather Dataset/Sunrise/sunrise2...,Lever du soleil,3200,1200
157,./Multi-class Weather Dataset/Cloudy/cloudy240...,Nuageux,300,138
682,./Multi-class Weather Dataset/Shine/shine25.jpg,Ensoleillé,852,480
224,./Multi-class Weather Dataset/Cloudy/cloudy300...,Nuageux,300,149


In [9]:
test_set.head()

,Picture Path,Label,width,height
412,./Multi-class Weather Dataset/Rain/rain20.jpg,Pluvieux,620,413
645,./Multi-class Weather Dataset/Shine/shine216.jpg,Ensoleillé,230,153
90,./Multi-class Weather Dataset/Cloudy/cloudy180...,Nuageux,261,175
717,./Multi-class Weather Dataset/Shine/shine53.jpg,Ensoleillé,284,177
393,./Multi-class Weather Dataset/Rain/rain183.jpg,Pluvieux,630,446


# Image Data generator

In [10]:
batch_size = 64
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
tr_gen = ImageDataGenerator()
ts_gen = ImageDataGenerator()

print("Train Set:")
train_gen = tr_gen.flow_from_dataframe(train_set, x_col='Picture Path', y_col='Label', target_size=img_size, class_mode='categorical',
                                       color_mode='rgb', shuffle=True, batch_size=batch_size)


print("Test Set:")
test_gen = ts_gen.flow_from_dataframe(test_set, x_col='Picture Path', y_col='Label', target_size=img_size, class_mode='categorical',
                                      color_mode='rgb', shuffle=False, batch_size=batch_size)


Train Set:
Found 900 validated image filenames belonging to 4 classes.
Test Set:
Found 225 validated image filenames belonging to 4 classes.


# Caracteristiques

In [11]:
def resize_and_crop_top(path, new_width=507, new_height=335):
    """
    Resize the image to the specified dimensions and keep only the top 70% of the image.
    
    Args:
    - image: numpy array representing the image.
    - new_width: desired width after resizing.
    - new_height: desired height after resizing.
    
    Returns:
    - Resized and cropped image.
    """
    image = io.imread(path)
    # Resize the image to the desired dimensions without interpolation
    resized_image = cv2.resize(image, (new_width, new_height))
    
    # Calculate the new height to keep only the top 70%
    cropped_height = int(new_height * 0.7)
    
    # Crop the image to keep only the top 70%
    cropped_image = resized_image[:cropped_height, :]
    
    return cropped_image

In [12]:

def resize_images(folder_path, target_size):

    # Parcourir toutes les images dans le dossier
    for filename in os.listdir(folder_path):
        if filename is not None:
            image_path = os.path.join(folder_path, filename)
            
            # Vérifier si le fichier est une image
            if os.path.isfile(image_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp',)) :
                
                # Charger l'image
                image = cv2.imread(image_path)

                # Vérifier si l'image est correctement chargée
                if image is not None:
                    # Redimensionner l'image à la nouvelle taille
                    resized_image = cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)
                    
                    # Enregistrer l'image redimensionnée
                    cv2.imwrite(image_path, resized_image)
                else:
                    print(f"Impossible de charger l'image : {image_path}")

    print("Toutes les images ont été redimensionnées avec succès.")

In [38]:
def get_luminance(image):
    # Check if the image has an alpha channel
    if image.shape[2] == 4:
        # Remove the alpha channel
        image = image[:, :, :3]
    gray_image = color.rgb2gray(image)
    luminance_mean = np.std(gray_image)
    return luminance_mean  


def get_histogram(image):
    red_channel = image[:, :, 0]
    green_channel = image[:, :, 1]
    blue_channel = image[:, :, 2]

    # Calculate marginal histograms
    red_hist, red_bins = np.histogram(red_channel.ravel(), bins=256)
    green_hist, green_bins = np.histogram(green_channel.ravel(), bins=256)
    blue_hist, blue_bins = np.histogram(blue_channel.ravel(), bins=256)

    # Calculate the means of each channel
    red_mean = np.average(red_bins[:-1], weights=red_hist)
    green_mean = np.average(green_bins[:-1], weights=green_hist)
    blue_mean = np.average(blue_bins[:-1], weights=blue_hist)

    return [red_mean, green_mean, blue_mean]


In [39]:
def calculate_lbp_histogram(image, n_points=8, radius=1, method='uniform'):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Calculate LBP patterns
    lbp = local_binary_pattern(image, n_points, radius, method)

    # Calculate the histogram of LBP patterns
    n_bins = int(lbp.max() + 1)
    hist, _ = np.histogram(lbp.ravel(), bins=n_bins, range=(0, n_bins))
    
    return hist.tolist()


In [45]:
# preprocess + process

def calculate_luminance(path):
    try:
        img = resize_and_crop_top(path)
        return get_luminance(img)
    except Exception as e:
        print(f"Error calculating luminance for image {path}: {str(e)}")
        return None

def calculate_histogram(path):
    try:
        img = resize_and_crop_top(path)
        return get_histogram(img)
    except Exception as e:
        print(f"Error calculating histogram for image {path}: {str(e)}")
        return None

def calculate_LBP(path):
    try:
        img = resize_and_crop_top(path)
        return calculate_lbp_histogram(img, n_points=8, radius=1, method='uniform')
    except Exception as e:
        print(f"Error calculating LBP for image {path}: {str(e)}")
        return None


In [46]:
# Save 

train_set['Luminance'] = None  
train_set['RGB'] = None
train_set['LBP'] = None

for i in range(len(train_set)):
    path = train_set['Picture Path'].iloc[i]
    lumin = calculate_luminance(path)
    hist = calculate_histogram(path)
    lbp = calculate_LBP(path)
    train_set['Luminance'].iloc[i] = lumin
    train_set['RGB'].iloc[i] = hist
    train_set['LBP'].iloc[i] = lbp


Error calculating luminance for image ./Multi-class Weather Dataset/Shine/shine131.jpg: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:3699: error: (-215:Assertion failed) !dsize.empty() in function 'cv::hal::resize'

Error calculating histogram for image ./Multi-class Weather Dataset/Shine/shine131.jpg: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:3699: error: (-215:Assertion failed) !dsize.empty() in function 'cv::hal::resize'

Error calculating LBP for image ./Multi-class Weather Dataset/Shine/shine131.jpg: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:3699: error: (-215:Assertion failed) !dsize.empty() in function 'cv::hal::resize'

Error calculating luminance for image ./Multi-class Weather Dataset/Cloudy/cloudy66.jpg: tuple index out of range
Error calculating histogram for image ./Multi-class Weather Dataset/Cloudy/cloudy66.jpg: too many indices for array: array i

In [58]:
test_set['Luminance'] = None  
test_set['RGB'] = None
test_set['LBP'] = None

for i in range(len(test_set)):
    path = test_set['Picture Path'].iloc[i]
    lumin = calculate_luminance(path)
    hist = calculate_histogram(path)
    lbp = calculate_LBP(path)
    test_set['Luminance'].iloc[i] = lumin
    test_set['RGB'].iloc[i] = hist
    test_set['LBP'].iloc[i] = lbp


Error calculating luminance for image ./Multi-class Weather Dataset/Cloudy/cloudy71.jpg: tuple index out of range
Error calculating histogram for image ./Multi-class Weather Dataset/Cloudy/cloudy71.jpg: too many indices for array: array is 2-dimensional, but 3 were indexed
Error calculating LBP for image ./Multi-class Weather Dataset/Cloudy/cloudy71.jpg: OpenCV(4.8.1) d:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.simd_helpers.hpp:92: error: (-2:Unspecified error) in function '__cdecl cv::impl::`anonymous-namespace'::CvtHelper<struct cv::impl::`anonymous namespace'::Set<3,4,-1>,struct cv::impl::A0xd8611878::Set<1,-1,-1>,struct cv::impl::A0xd8611878::Set<0,2,5>,3>::CvtHelper(const class cv::_InputArray &,const class cv::_OutputArray &,int)'
> Invalid number of channels in input image:
>     'VScn::contains(scn)'
> where
>     'scn' is 1



In [59]:
train_set = train_set.dropna()
test_set = test_set.dropna()
#train_set.to_csv('train_set.csv', index=False)
#test_set.to_csv('test_set.csv', index=False)

In [54]:
train_set.head()

,Picture Path,Label,width,height,Luminance,RGB,LBP
646,./Multi-class Weather Dataset/Shine/shine217.jpg,Ensoleillé,270,169,0.222895,"[65.78350459354296, 123.6077803380873, 157.317...","[975, 3516, 2790, 15257, 27956, 28952, 7851, 7..."
892,./Multi-class Weather Dataset/Sunrise/sunrise2...,Lever du soleil,3200,1200,0.192201,"[124.46965440562889, 63.85506010800713, 28.894...","[7105, 8965, 4416, 10314, 13292, 17099, 8923, ..."
157,./Multi-class Weather Dataset/Cloudy/cloudy240...,Nuageux,300,138,0.093878,"[108.03152604983227, 108.03152604983227, 108.0...","[255, 1697, 1105, 13024, 31898, 35377, 7565, 6..."
682,./Multi-class Weather Dataset/Shine/shine25.jpg,Ensoleillé,852,480,0.21004,"[100.06668411075077, 131.0842690843153, 167.38...","[2497, 4193, 1729, 11196, 11722, 36659, 11868,..."
224,./Multi-class Weather Dataset/Cloudy/cloudy300...,Nuageux,300,149,0.243915,"[147.60495053097446, 163.75665245852932, 187.3...","[437, 2155, 1398, 12525, 25460, 31878, 7130, 6..."


# Classifier 

In [57]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

X_train_rgb = np.array(train_set['RGB'].values.tolist())
X_train_lbp = np.array(train_set['LBP'].values.tolist())
X_train_luminance = np.array(train_set['Luminance'].values.tolist())

X_train = np.column_stack((X_train_rgb, X_train_lbp, X_train_luminance))
y_train = train_set['Label']

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [60]:
X_test_rgb = np.array(test_set['RGB'].values.tolist())
X_test_lbp = np.array(test_set['LBP'].values.tolist())
X_test_luminance = np.array(test_set['Luminance'].values.tolist())

X_test = np.column_stack((X_test_rgb, X_test_lbp, X_test_luminance))
y_test = test_set['Label']

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.6517857142857143


In [61]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", f1)


F1 Score: 0.6569408400143437
